# The Key to Life

In [161]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [113]:
df = pd.read_csv("brian_keylogs_clean.csv")
el = pd.read_csv("eleanor_keylogs_clean.csv")

In [123]:
def featurize_that_bish(df, top_pairs=None):
    data = pd.DataFrame(df.groupby("pair")['delta'].mean().reset_index())
    if top_pairs:
        data = data.where(data["pair"].isin(top_pairs)).dropna()
    data.columns = ["pair", "delta_avg"]
    data = data.reset_index()
    del data["index"]
    data = data.sort_values("pair", ascending=False)
    return data.transpose()

In [147]:
top_pairs = set(df.groupby("pair").count().sort_values("delta", ascending=False).reset_index()[:100]['pair'])
df = shuffle(df)
el = shuffle(el)
bf_train, bf_test = train_test_split(df, test_size = 0.33)
bf_train, bf_test = featurize_that_bish(bf_train, top_pairs), featurize_that_bish(bf_test, top_pairs)
el_train, el_test = train_test_split(el, test_size = 0.33)
el_train, el_test = featurize_that_bish(el_train, top_pairs), featurize_that_bish(el_test, top_pairs)

In [148]:
def combine_data(a, b):
    new = pd.DataFrame(a)
    new.append(b, axis=1)
    return new

In [159]:
elt = el_train.iloc[1:]
bft = bf_train.iloc[1:]
elte = el_test.iloc[1:]
bfte = bf_test.iloc[1:]
train = np.asmatrix(bft.append(elt))
test = np.asmatrix(bfte.append(elte))
y = np.array([0, 1])

In [162]:
log = LogisticRegression()
log.fit(train, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

array([0, 1])